In [13]:
from tqdm import tqdm
import os
import re
import json
import pdfplumber

In [14]:
file_name = '탄약 수송 결박지침서'
folder_path = './data/jisi/' 
output_path = './'

In [15]:
## 제거해야할 페이지 설정
drop_page = list(range(1,8)) 
        # [23] + \
        # list(range(128,141))
drop_page = [p-1 for p in drop_page]

In [16]:
## 제거할 페이지 남기고 txt 추출
with pdfplumber.open(folder_path + file_name + '.pdf') as pdf:
    text = ''
    for page_number in tqdm(range(len(pdf.pages))):
        if page_number not in drop_page:
            ## 페이지 텍스트 추출
            text += pdf.pages[page_number].extract_text()

            # 텍스트 파일로 저장
        with open(output_path + file_name + '.txt', 'w', encoding='utf-8') as txt_file:
            txt_file.write(text)

# 텍스트를 \n 기준으로 리스트화
text_list = [line + '\n' for line in text.split('\n') if line]

100%|██████████| 147/147 [00:29<00:00,  5.03it/s]


In [93]:
class ContentsProcess:
    def __init__(self):
        pass

    def convert_key_to_num(contents, number_key, title_type):
        ## 특정 문자가 왔을 때 이걸 차례로 넘버링하기
        # number_key = '¢'

        chapter_number = 1
        result = []
        for line in contents:
            if line.startswith(number_key):
                if title_type != '조':
                    line = f'제{chapter_number}{title_type} {line[2:-1]}\n'
                else:
                    line = f'제{chapter_number}{title_type}({line[2:-1]})\n'
                chapter_number += 1
            result.append(line)
        
        return result
    
    def convert_num_to_jo(contents, title_type):
        ## 1. -> 제1조() 로 변환해주기
        refined_contents = []
        pattern = re.compile(r'^(\s*)(\d+)\.(\s+)(.+)\n')
        
        for line in contents:
            match = pattern.match(line)
            if match:
                etc = match.group(1)
                number = match.group(2)
                content = match.group(4)
                if title_type != '조':
                    refined_line = f"{etc}제{number}{title_type} {content}\n"
                    refined_contents.append(refined_line)
                else:
                    refined_line = f"{etc}제{number}{title_type}({content})\n"
                    refined_contents.append(refined_line)
            else:
                refined_contents.append(line)
        
        return refined_contents
    
    

    def convert_rom_to_num(contents, title_type):
        ## 로마숫자를 제1조()로 바꾸기
        def roman_to_int(roman):
            ## 로마숫자 아라비아 숫자로 변환
            roman_numerals = {
                'Ⅰ': 1, 'Ⅱ': 2, 'Ⅲ': 3, 'Ⅳ': 4, 'Ⅴ': 5, 'Ⅵ': 6, 'Ⅶ': 7, 'Ⅷ': 8, 'Ⅸ': 9, 'Ⅹ': 10,
                'I': 1, 'II': 2, 'III': 3, 'IV': 4, 'V': 5, 'VI': 6, 'VII': 7, 'VIII': 8, 'IX': 9, 'X': 10
            }
            return roman_numerals.get(roman, 0)
        pattern = re.compile(r'^(Ⅰ|Ⅱ|Ⅲ|Ⅳ|Ⅴ|Ⅵ|Ⅶ|Ⅷ|Ⅸ|Ⅹ|I{1,3}|IV|V|VI{0,3}|IX|X)[\.\s]')
        
        def replace_line(line, title_type):
            match = pattern.match(line)
            if match:
                num = roman_to_int(match.group(1))
                title = line[match.end():].strip()
                if title_type != '조':
                    return f'제{num}{title_type} {title}\n'
                else:
                    return f'제{num}{title_type}({title})\n'
            return line
        
        return [replace_line(line,title_type) for line in contents]
    
    def convert_kor_to_num(contents, title_type):
        # 한글 목차 항목을 찾기 위한 정규 표현식
        pattern = re.compile(r'^\s*([가-하])\.\s*(.+)\n')
        
        # 현재 조항 번호
        article_number = 1
        
        # 변환된 텍스트를 저장할 리스트
        converted_lines = []
        
        for line in contents:
            # 한글 목차 항목을 찾기 위한 정규 표현식 매칭 및 변환
            if title_type != '조':
                new_line = pattern.sub(lambda m: f'제{article_number}{title_type} {m.group(2)}\n', line)
            else:
                new_line = pattern.sub(lambda m: f'제{article_number}{title_type}({m.group(2)})\n', line)
            if new_line != line:  # 변환이 일어난 경우에만 조항 번호 증가
                article_number += 1
            converted_lines.append(new_line)
        
        return converted_lines

In [99]:
contents = [
    "Ⅰ. 첫번째 내용\n",
    "Ⅱ. 두번째 내용\n",
    "Ⅲ. 세번째 내용\n",
    "기타 본문 내용\n",
    "Ⅳ. 네번째 내용\n",
    "XII. 열두번째 내용",
    "I. 또 다른 첫번째 내용",
    "II. 또 다른 두번째 내용",
    "XI. 열한번째 내용",
    "L. 오십번째 내용",
    "XC. 아흔번째 내용",
    "C. 백번째 내용"
]

In [100]:
ContentsProcess.convert_rom_to_num(contents,'항')

['제1항 첫번째 내용',
 '제2항 두번째 내용',
 '제3항 세번째 내용',
 '기타 본문 내용\n',
 '제4항 네번째 내용',
 'XII. 열두번째 내용',
 '제1항 또 다른 첫번째 내용',
 '제2항 또 다른 두번째 내용',
 'XI. 열한번째 내용',
 'L. 오십번째 내용',
 'XC. 아흔번째 내용',
 'C. 백번째 내용']